<h1><span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pytorch-Demo" data-toc-modified-id="Pytorch-Demo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pytorch Demo</a></span></li><li><span><a href="#Onnx-Demo" data-toc-modified-id="Onnx-Demo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Onnx Demo</a></span></li><li><span><a href="#Onnx-Model-File-Analysis" data-toc-modified-id="Onnx-Model-File-Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Onnx Model File Analysis</a></span><ul class="toc-item"><li><span><a href="#Modify-initializer-(Tensor)" data-toc-modified-id="Modify-initializer-(Tensor)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Modify initializer (Tensor)</a></span></li><li><span><a href="#Modify-node-(Attributes)" data-toc-modified-id="Modify-node-(Attributes)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Modify node (Attributes)</a></span></li></ul></li><li><span><a href="#Runtime" data-toc-modified-id="Runtime-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Runtime</a></span></li><li><span><a href="#References" data-toc-modified-id="References-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>References</a></span></li></ul></div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%watermark -v -p tensorflow,tensorboard
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri).text
                if x.status == 200:
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.5
sklearn 0.24.2
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.3
PIL 8.2.0
matplotlib 3.3.4
CPython 3.6.9
IPython 7.16.1

torch 1.9.0
torchvision 0.10.0
torchaudio not installed
CPython 3.6.9
IPython 7.16.1

tensorflow 2.6.0
tensorboard 2.6.0


In [2]:
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'tensorboard-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_cell_magic
def template_writefile(line, cell):
    with open(line, 'w') as fw:
        fw.write(cell.format(**globals()))

@register_line_magic
def netron(line):
    args = line.split()
    file, port, height = args[0], int(args[1]), 600
    if len(args) == 3:
        height = int(args[2])
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #    pid = int(res[0].split(' ')[1])
    #    !kill -9 $pid
    import netron
    netron.start(file, address=('0.0.0.0', port), browse=False)
    display_html(port, height)


In [3]:

###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')


In [4]:
import onnx
import onnx.helper as OH
import onnxruntime as rt
from torchvision import datasets
from PIL import Image
from collections import OrderedDict

## Pytorch Demo

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TorchModel(nn.Module):
    def __init__(self):
        super(TorchModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(32, eps=1e-5)
        self.relu1 = nn.ReLU()
        self.avgpool1 = nn.AdaptiveAvgPool2d((1, 1))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=10, kernel_size=(1, 1), padding=(0, 0))
        self.flatten = nn.Flatten()
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.avgpool1(x)
        x = self.conv2(x)
        return self.flatten(x)
model = TorchModel().to(device)
    
# model = nn.Sequential(
#     OrderedDict(
#         [
#             ('conv1', nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=(1, 1))),
#             ('bn1', nn.BatchNorm2d(32, eps=1e-5)),
#             ('relu1', nn.ReLU()),
#             ('avgpool1', nn.AdaptiveAvgPool2d((1, 1))),
#             ('conv2', nn.Conv2d(in_channels=32, out_channels=10, kernel_size=(1, 1), padding=(0, 0))),
#             ('flatten', nn.Flatten())
#         ]
#     )
# )

for name, param in model.named_parameters():
    print(name)

conv1.weight
conv1.bias
bn1.weight
bn1.bias
conv2.weight
conv2.bias


/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
train_dataset = datasets.MNIST(
    root='/data/nb_data/datasets/', train=True, download=False,
    transform=T.Compose([T.Lambda(lambda x: x.convert('RGB')), T.ToTensor(), T.Normalize((0.1307,), (0.3081,))]))
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = O.Adam(model.parameters(), lr=1e-4)
# not learning, only for onnx
for epoch in range(1):
    for image, target in train_loader:
        image, target = image.to(device), target.to(device)
        output = model(image)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break

In [7]:
torch.onnx.export(
    model, torch.randn((1, 3, 28, 28)),
    '/data/nb_data/tmp/torch_model.onnx', 
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input' : {0 : 'BS'}, 'output' : {0 : 'BS'}},
    do_constant_folding=True, # TODO? can not show layer real name
    verbose=True)

graph(%input : Float(*, 3, 28, 28, strides=[2352, 784, 28, 1], requires_grad=0, device=cpu),
      %conv2.weight : Float(10, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cpu),
      %conv2.bias : Float(10, strides=[1], requires_grad=1, device=cpu),
      %17 : Float(32, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %18 : Float(32, strides=[1], requires_grad=0, device=cpu)):
  %16 : Float(*, 32, 28, 28, strides=[25088, 784, 28, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input, %17, %18)
  %12 : Float(*, 32, 28, 28, strides=[25088, 784, 28, 1], requires_grad=1, device=cpu) = onnx::Relu(%16) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1298:0
  %13 : Float(*, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cpu) = onnx::GlobalAveragePool(%12) # /usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1130:0
  %14 : Float(*, 10, 1, 1, stri

In [8]:
%netron /data/nb_data/tmp/torch_model.onnx 8228 600

Serving '/data/nb_data/tmp/torch_model.onnx' at http://0.0.0.0:8228


## Onnx Demo

<img src="https://gitee.com/qrsforever/blog_source_assets/raw/master/Common/onnx_proto.gif" width=600px />

In [9]:
# Input and Output ValueInfoProto

X = OH.make_tensor_value_info(
    'X',
    onnx.TensorProto.FLOAT,
    [1, 3, 28, 28])

Y = OH.make_tensor_value_info(
    'Y',
    onnx.TensorProto.FLOAT,
    [1, 10])


# Conv1: NodeProto

conv1_W = np.ones(shape=(32, 3, 3, 3)).astype(np.float32)
conv1_W_tensor = OH.make_tensor(
    name='conv1_W',
    data_type=onnx.TensorProto.FLOAT,
    dims=conv1_W.shape,
    vals=conv1_W.flatten().tolist())

conv1_B = np.ones(shape=(32)).astype(np.float32)
conv1_B_tensor = OH.make_tensor(
    name='conv1_B',
    data_type=onnx.TensorProto.FLOAT,
    dims=conv1_B.shape,
    vals=conv1_B.flatten().tolist())

conv1_node = OH.make_node(
    name="conv1",
    op_type="Conv",
    inputs=['X', 'conv1_W', 'conv1_B'],
    outputs=['conv1_Y'],
    kernel_shape=(3, 3),
    pads=(1, 1, 1, 1),
)


# Bn1: NodeProto

bn1_params = OrderedDict({
    name: np.random.randn(32).astype(np.float32) for name in ('bn1_scale', 'bn1_bias', 'bn1_mean', 'bn1_var')
})

bn1_initializer_tensor = {
    f'{key}_tensor': OH.make_tensor(
        name=f'{key}_tensor',
        data_type=onnx.TensorProto.FLOAT,
        dims=val.shape,
        vals=val.flatten().tolist()) for key, val in bn1_params.items()
}

bn1_node = OH.make_node(
    name="bn1",
    op_type="BatchNormalization",
    inputs=[
        'conv1_Y', *bn1_initializer_tensor.keys()
    ],
    outputs=['bn1_Y'],
    epsilon=1e-6
)

# Create a ReLU node (NodeProto).

relu1_node = OH.make_node(
    name="relu1",
    op_type="Relu",
    inputs=['bn1_Y'],
    outputs=['relu1_Y'],
)

# Create a Avg Pool node (NodeProto)

avg_pool1_node = onnx.helper.make_node(
    name="avgpool1",
    op_type="GlobalAveragePool",
    inputs=['relu1_Y'],
    outputs=['avgpool1_Y'],
)


# Conv2: NodeProto

conv2_W = np.ones(shape=(10, 32, 1, 1)).astype(np.float32)
conv2_W_tensor = OH.make_tensor(
    name='conv2_W',
    data_type=onnx.TensorProto.FLOAT,
    dims=conv2_W.shape,
    vals=conv2_W.flatten().tolist())

conv2_B = np.ones(shape=(10)).astype(np.float32)
conv2_B_tensor = OH.make_tensor(
    name='conv2_B',
    data_type=onnx.TensorProto.FLOAT,
    dims=conv2_B.shape,
    vals=conv2_B.flatten().tolist())

conv2_node = OH.make_node(
    name="conv2",
    op_type="Conv",
    inputs=['avgpool1_Y', 'conv2_W', 'conv2_B'],
    outputs=['conv2_Y'],
    kernel_shape=(1, 1),
    pads=(0, 0, 0, 0)
)


# Flatten: NodeProto

flatten_node = OH.make_node(
    name='flatten',
    op_type='Flatten',
    inputs=['conv2_Y'],
    outputs=['Y'],
    axis=1
)


# Create the graph (GraphProto)

graph_def = OH.make_graph(
    nodes=[conv1_node, bn1_node, relu1_node, avg_pool1_node, conv2_node, flatten_node],
    name="ConvNet",
    inputs=[X],
    outputs=[Y],
    initializer=[
        conv1_W_tensor, conv1_B_tensor,
        *bn1_initializer_tensor.values(),
        conv2_W_tensor, conv2_B_tensor
    ]
)


# Create the model (ModelProto)

model_def = OH.make_model(graph_def, producer_name="onnx-example")
model_def.opset_import[0].version = 13
model_def = onnx.shape_inference.infer_shapes(model_def)

onnx.checker.check_model(model_def)
onnx.save(model_def, "/data/nb_data/tmp/convnet.onnx")

In [10]:
%netron /data/nb_data/tmp/convnet.onnx 8229 300

Serving '/data/nb_data/tmp/convnet.onnx' at http://0.0.0.0:8229


## Onnx Model File Analysis

In [11]:
model = onnx.load("/data/nb_data/tmp/convnet.onnx")
onnx.checker.check_model(model)
graph_def = model.graph

In [12]:
model.model_version, model.opset_import, \
model.producer_name, model.producer_version, model.training_info

(0,
 [version: 13
 ],
 'onnx-example',
 '',
 [])

In [13]:
# Node Out ValueInfo (xxx_Y)
graph_def.value_info

[name: "conv1_Y"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 32
      }
      dim {
        dim_value: 28
      }
      dim {
        dim_value: 28
      }
    }
  }
}
, name: "bn1_Y"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 32
      }
      dim {
        dim_value: 28
      }
      dim {
        dim_value: 28
      }
    }
  }
}
, name: "relu1_Y"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 32
      }
      dim {
        dim_value: 28
      }
      dim {
        dim_value: 28
      }
    }
  }
}
, name: "avgpool1_Y"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 32
      }
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 1
      }
    }
  }
}
, name

In [14]:
graph_def.node

[input: "X"
input: "conv1_W"
input: "conv1_B"
output: "conv1_Y"
name: "conv1"
op_type: "Conv"
attribute {
  name: "kernel_shape"
  ints: 3
  ints: 3
  type: INTS
}
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}
, input: "conv1_Y"
input: "bn1_scale_tensor"
input: "bn1_bias_tensor"
input: "bn1_mean_tensor"
input: "bn1_var_tensor"
output: "bn1_Y"
name: "bn1"
op_type: "BatchNormalization"
attribute {
  name: "epsilon"
  f: 9.999999974752427e-07
  type: FLOAT
}
, input: "bn1_Y"
output: "relu1_Y"
name: "relu1"
op_type: "Relu"
, input: "relu1_Y"
output: "avgpool1_Y"
name: "avgpool1"
op_type: "GlobalAveragePool"
, input: "avgpool1_Y"
input: "conv2_W"
input: "conv2_B"
output: "conv2_Y"
name: "conv2"
op_type: "Conv"
attribute {
  name: "kernel_shape"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "pads"
  ints: 0
  ints: 0
  ints: 0
  ints: 0
  type: INTS
}
, input: "conv2_Y"
output: "Y"
name: "flatten"
op_type: "Flatten"
attribute {
  name: "axis"
  i:

### Modify initializer (Tensor)

In [15]:
for initializer in graph_def.initializer:
    print(f"Tensor Name: {initializer.name}, Data Type: {initializer.data_type}, Shape: {initializer.dims}")
    if initializer.data_type == onnx.TensorProto.FLOAT:
        print('Before Change:')
        print(initializer.float_data[:10])
        for i in range(np.prod(initializer.dims)):
            initializer.float_data[i] *= 2
        print('After Change:')
        print(initializer.float_data[:10])

Tensor Name: conv1_W, Data Type: 1, Shape: [32, 3, 3, 3]
Before Change:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
After Change:
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
Tensor Name: conv1_B, Data Type: 1, Shape: [32]
Before Change:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
After Change:
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
Tensor Name: bn1_scale_tensor, Data Type: 1, Shape: [32]
Before Change:
[0.16064798831939697, 0.32743120193481445, 0.8825830221176147, -1.2554028034210205, -0.767731249332428, -0.2491602599620819, -0.284347265958786, 0.2278527468442917, 1.0320441722869873, -0.3268558382987976]
After Change:
[0.32129597663879395, 0.6548624038696289, 1.7651660442352295, -2.510805606842041, -1.535462498664856, -0.4983205199241638, -0.568694531917572, 0.4557054936885834, 2.0640883445739746, -0.6537116765975952]
Tensor Name: bn1_bias_tensor, Data Type: 1, Shape: [32]
Before Change:
[0.16661114990711212, 0.0127743324264884, -1.5268645286560059, -0.

### Modify node (Attributes)

In [16]:
for node in graph_def.node:
    if node.op_type == "BatchNormalization":
        print("Attributes before adding:")
        for i, attribute in enumerate(node.attribute):
            print(attribute)
            if attribute.name == 'epsilon':
                node.attribute.remove(attribute)
        epsilon_attribute = OH.make_attribute("epsilon", 0.0001)
        node.attribute.extend([epsilon_attribute])
        print("Attributes after adding:")
        for attribute in node.attribute:
            print(attribute)

Attributes before adding:
name: "epsilon"
f: 9.999999974752427e-07
type: FLOAT

Attributes after adding:
name: "epsilon"
f: 9.999999747378752e-05
type: FLOAT



In [17]:
onnx.checker.check_model(model)
onnx.save(model, "/data/nb_data/tmp/convnet_modified.onnx")

In [18]:
%netron /data/nb_data/tmp/convnet_modified.onnx 8227 300

Serving '/data/nb_data/tmp/convnet_modified.onnx' at http://0.0.0.0:8227


## Runtime

In [19]:
sess = rt.InferenceSession('/data/nb_data/tmp/torch_model.onnx')  
inputs = {sess.get_inputs()[0].name: np.random.randn(1, 3, 28, 28).astype('float32')}
outputs = [sess.get_outputs()[0].name]
predictions = sess.run(outputs, inputs)
predictions

[array([[ 0.24498022,  0.02249821, -0.22628897, -0.17308606, -0.0756798 ,
          0.28872386, -0.23748004,  0.13234285, -0.28075558, -0.0848928 ]],
       dtype=float32)]

## References

- [Creating and Modifying ONNX Model Using ONNX Python API][1]

- [Onnx Operator Schemas][2]

[1]: https://leimao.github.io/blog/ONNX-Python-API/
[2]: https://github.com/onnx/onnx/blob/master/docs/Operators.md